In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
cd /gdrive/MyDrive/182/procgen/christian

/gdrive/MyDrive/182/procgen/christian


In [3]:
ls

Analysis2.ipynb     logs/         train_colab.ipynb    train.py
Analysis.ipynb      old_scripts/  train_colab.py
CustomCNNPolicy.py  __pycache__/  train_new.py
GymDebug.ipynb      test.py       train_procgen_pfrl/


In [4]:
! pip install procgen stable-baselines3[extra]

     |████████████████████████████████| 39.9MB 137kB/s 
     |████████████████████████████████| 153kB 63.7MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 204kB 49.0MB/s 
     |████████████████████████████████| 22.2MB 1.1MB/s 
     |████████████████████████████████| 675kB 65.3MB/s 
     |████████████████████████████████| 3.3MB 63.9MB/s 
     |████████████████████████████████| 40kB 6.7MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [5]:
from procgen import ProcgenEnv
from train_procgen_pfrl.vec_env import VecExtractDictObs
from train_procgen_pfrl.vec_env import VecMonitor
from train_procgen_pfrl.vec_env import VecNormalize

In [ ]:
parser = argparse.ArgumentParser(
    description='Process procgen training arguments.')

# Experiment parameters.
parser.add_argument(
    '--distribution-mode', type=str, default='easy',
    choices=['easy', 'hard', 'exploration', 'memory', 'extreme'])
parser.add_argument('--env-name', type=str, default='starpilot')
parser.add_argument('--num-envs', type=int, default=64)
parser.add_argument('--num-levels', type=int, default=0)
parser.add_argument('--start-level', type=int, default=0)
parser.add_argument('--num-threads', type=int, default=4)
parser.add_argument('--exp-name', type=str, default='trial01')
parser.add_argument('--log-dir', type=str, default='./log')
parser.add_argument('--model-file', type=str, default=None)
parser.add_argument('--method-label', type=str, default='vanilla')

# PPO parameters.
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--lr', type=float, default=5e-4)
parser.add_argument('--ent-coef', type=float, default=0.01)
parser.add_argument('--vf-coef', type=float, default=0.5)
parser.add_argument('--gamma', type=float, default=0.999)
parser.add_argument('--lam', type=float, default=0.95)
parser.add_argument('--clip-range', type=float, default=0.2)
parser.add_argument('--max-grad-norm', type=float, default=0.5)
parser.add_argument('--nsteps', type=int, default=256)
parser.add_argument('--batch-size', type=int, default=8)
parser.add_argument('--nepochs', type=int, default=3)
parser.add_argument('--max-steps', type=int, default=25_000_000)
parser.add_argument('--save-interval', type=int, default=100)

configs = parser.parse_args(
    args=[
          
    ]
)

In [ ]:
def create_venv(config, is_valid=False):
    venv = ProcgenEnv(
        num_envs=config.num_envs,
        env_name=config.env_name,
        num_levels=0 if is_valid else config.num_levels,
        start_level=0 if is_valid else config.start_level,
        distribution_mode=config.distribution_mode,
        num_threads=config.num_threads,
    )
    venv = VecExtractDictObs(venv, "rgb")
    venv = VecMonitor(venv=venv, filename=None, keep_buf=100)
    return VecNormalize(venv=venv, ob=False)